In [ ]:
from typing import Callable

with open('17-test.txt', 'r') as f:
  data: str = f.read()

raw_registers: str
raw_program: str

raw_registers, raw_program = data.split('\n\n')
r: list[int] = [int(''.join(filter(str.isdigit, x))) for x in raw_registers.split('\n')]

registers: dict[str, int] = { 'A': r[0], 'B': r[1], 'C': r[2] }
program: list[int] = [int(x) for x in raw_program.split(' ')[1].split(',')]

combo: dict[int, int] = {
  0: 0,
  1: 1,
  2: 2, 
  3: 3, 
  4: r[0],
  5: r[1],
  6: r[2],
}

def adv(o: list[int], p: int, r: dict[str, int], n: int) -> tuple[int, list[int], dict[str, int]]: 
  r['A'] = int(r['A']/(2**(combo[n])))
  combo[4] = r['A']
  return p + 2, o, r

def bxl(o: list[int], p: int, r: dict[str, int], n: int) -> tuple[int, list[int], dict[str, int]]: 
  r['B'] = r['B'] ^ n
  combo[5] = r['B']
  return p + 2, o, r

def bst(o: list[int], p: int, r: dict[str, int], n: int) -> tuple[int, list[int], dict[str, int]]: 
  r['B'] = combo[n] % 8
  combo[5] = r['B']
  return p + 2, o, r

def jnz(o: list[int], p: int, r: dict[str, int], n: int) -> tuple[int, list[int], dict[str, int]]:
  return p + 2 if r['A'] == 0 else n, o, r

def bxc(o: list[int], p: int, r: dict[str, int], _: int) -> tuple[int, list[int], dict[str, int]]:
  r['B'] = r['B'] ^ r['C']
  combo[5] = r['B']
  return p + 2, o, r

def out(o: list[int], p: int, r: dict[str, int], n: int) -> tuple[int, list[int], dict[str, int]]:
  return p + 2, [*o, (combo[n] % 8)], r

def bdv(o: list[int], p: int, r: dict[str, int], n: int) -> tuple[int, list[int], dict[str, int]]:
  r['B'] = int(r['A']/(2**(combo[n])))
  combo[5] = r['B']
  return p + 2, o, r

def cdv(o: list[int], p: int, r: dict[str, int], n: int) -> tuple[int, list[int], dict[str, int]]:
  r['C'] = int(r['A']/(2**(combo[n])))
  combo[6] = r['C']
  return p + 2, o, r

instructions: dict[int, Callable] = {
  0: adv,
  1: bxl,
  2: bst,
  3: jnz,
  4: bxc,
  5: out,
  6: bdv,
  7: cdv
}

pointer: int = 0
output: list[int] = []
while pointer < len(program)-1:
  pointer, output, registers = instructions[program[pointer]](output, pointer, registers, program[pointer+1])
  
output: str = ','.join(map(str, output)) 
output


In [ ]:
from typing import Callable

with open('17.txt', 'r') as f: data: str = f.read()

raw_registers: str
raw_program: str

raw_registers, raw_program = data.split('\n\n')
r: list[int] = [int(''.join(filter(str.isdigit, x))) for x in raw_registers.split('\n')]
program: list[int] = [int(x) for x in raw_program.split(' ')[1].split(',')]
registers: dict[str, int] = { 'A': r[0], 'B': r[1], 'C': r[2] }

combo: dict[int, int] = {
  0: 0,
  1: 1,
  2: 2, 
  3: 3, 
  4: registers['A'],
  5: registers['B'],
  6: registers['C'],
}

def adv(output: str, pointer: int, operand: int) -> tuple[int, str]: 
  registers['A'] = int(registers['A']/(2**(combo[operand])))
  combo[4] = registers['A']
  return pointer + 2, output

def bxl(output: str, pointer: int, operand: int) -> tuple[int, str]: 
  registers['B'] = registers['B'] ^ operand
  combo[5] = registers['B']
  return pointer + 2, output

def bst(output: str, pointer: int, operand: int) -> tuple[int, str]: 
  registers['B'] = combo[operand] % 8
  combo[5] = registers['B']
  return pointer + 2, output

def jnz(output: str, pointer: int, operand: int) -> tuple[int, str]:
  return pointer + 2 if registers['A'] == 0 else operand, output

def bxc(output: str, pointer: int, _: int) -> tuple[int, str]:
  registers['B'] = registers['B'] ^ registers['C']
  combo[5] = registers['B']
  return pointer + 2, output

def out(output: str, pointer: int, operand: int) -> tuple[int, str]:
  return pointer + 2, output + str(combo[operand]%8) + ',' 

def bdv(output: str, pointer: int, operand: int) -> tuple[int, str]:
  registers['B'] = int(registers['A']/(2**(combo[operand])))
  combo[5] = registers['B']
  return pointer + 2, output

def cdv(output: str, pointer: int, operand: int) -> tuple[int, str]:
  registers['C'] = int(registers['A']/(2**(combo[operand])))
  combo[6] = registers['C']
  return pointer + 2, output

instructions: dict[int, Callable] = {
  0: adv, 1: bxl, 2: bst, 3: jnz, 4: bxc, 5: out, 6: bdv, 7: cdv
}

prog_str: str = ','.join(map(str, program)) 
i: int = 0
while True:
  registers = { 'A': i, 'B': r[1], 'C': r[2] }
  combo[4] = registers['A']
  combo[5] = registers['B']
  combo[6] = registers['C']
  pointer: int = 0
  output: str = ''

  while pointer < len(program)-1:
    pointer, output = instructions[program[pointer]](output, pointer, program[pointer+1])
  if prog_str == output[0:-1]: break 
  i += 1

print(output)
print(i)

KeyboardInterrupt: 